In [120]:
import tensorflow as tf
import pandas as pd
import calendar;
import time;

In [165]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, Imputer

In [123]:
books = pd.read_csv(r"../Dataset/BX-Books.csv", sep=";", encoding='latin1')
ratings = pd.read_csv(r"../Dataset/BX-Book-Ratings.csv", sep=";",
                      encoding='latin1')
users = pd.read_csv(r"../Dataset/BX-Users.csv", sep=";",
                    encoding='latin1')

In [124]:
full_data_df = ratings.merge(books,on='ISBN').merge(users,on='User-ID')

In [125]:
full_data_df.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Location,Age
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa",NaN
1,2313,034545104X,5,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,"cincinnati, ohio, usa",23.0
2,2313,0812533550,9,Ender's Game (Ender Wiggins Saga (Paperback)),Orson Scott Card,1986,Tor Books,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,http://images.amazon.com/images/P/0812533550.0...,"cincinnati, ohio, usa",23.0
3,2313,0679745580,8,In Cold Blood (Vintage International),TRUMAN CAPOTE,1994,Vintage,http://images.amazon.com/images/P/0679745580.0...,http://images.amazon.com/images/P/0679745580.0...,http://images.amazon.com/images/P/0679745580.0...,"cincinnati, ohio, usa",23.0
4,2313,0060173289,9,Divine Secrets of the Ya-Ya Sisterhood : A Novel,Rebecca Wells,1996,HarperCollins,http://images.amazon.com/images/P/0060173289.0...,http://images.amazon.com/images/P/0060173289.0...,http://images.amazon.com/images/P/0060173289.0...,"cincinnati, ohio, usa",23.0


In [173]:
full_data_df.columns

Index(['User-ID', 'ISBN', 'Book-Rating', 'Book-Title', 'Book-Author',
       'Year-Of-Publication', 'Publisher', 'Image-URL-S', 'Image-URL-M',
       'Image-URL-L', 'Location', 'Age'],
      dtype='object')

In [174]:
useful_data_df = full_data_df[['Location', 'Age', 'Book-Author', 'Year-Of-Publication', 'Book-Rating']]
imp = Imputer(copy = False)
imp.fit_transform(useful_data_df['Age'].values.reshape(-1, 1))
useful_data_df.head()

,Location,Age,Book-Author,Year-Of-Publication,Book-Rating
0,"tyler, texas, usa",37.397719,M. J. Rose,2002,0
1,"cincinnati, ohio, usa",23.000000,M. J. Rose,2002,5
2,"cincinnati, ohio, usa",23.000000,Orson Scott Card,1986,9
3,"cincinnati, ohio, usa",23.000000,TRUMAN CAPOTE,1994,8
4,"cincinnati, ohio, usa",23.000000,Rebecca Wells,1996,9


In [175]:
loc_df = pd.DataFrame([ x.split(', ', 2)[0:2] for x in useful_data_df['Location'].tolist() ])
loc_df.columns = ['City', 'State']
useful_data_df = pd.concat([useful_data_df, loc_df], axis=1).drop('Location', axis=1)
useful_data_df.head()

,Age,Book-Author,Year-Of-Publication,Book-Rating,City,State
0,37.397719,M. J. Rose,2002,0,tyler,texas
1,23.000000,M. J. Rose,2002,5,cincinnati,ohio
2,23.000000,Orson Scott Card,1986,9,cincinnati,ohio
3,23.000000,TRUMAN CAPOTE,1994,8,cincinnati,ohio
4,23.000000,Rebecca Wells,1996,9,cincinnati,ohio


In [176]:
le = LabelEncoder()
useful_data_df['Book-Author-Enc'] = le.fit_transform(useful_data_df['Book-Author'].values).reshape(-1,1)

In [177]:
le = LabelEncoder()
useful_data_df['City-Enc'] = le.fit_transform(useful_data_df['City'].values).reshape(-1,1)

In [178]:
le = LabelEncoder()
useful_data_df['State-Enc'] = le.fit_transform(useful_data_df['State'].values).reshape(-1,1)

In [179]:
useful_data_df.head()

,Age,Book-Author,Year-Of-Publication,Book-Rating,City,State,Book-Author-Enc,City-Enc,State-Enc
0,37.397719,M. J. Rose,2002,0,tyler,texas,61695,13340,1839
1,23.000000,M. J. Rose,2002,5,cincinnati,ohio,61695,2631,1397
2,23.000000,Orson Scott Card,1986,9,cincinnati,ohio,73676,2631,1397
3,23.000000,TRUMAN CAPOTE,1994,8,cincinnati,ohio,93473,2631,1397
4,23.000000,Rebecca Wells,1996,9,cincinnati,ohio,80512,2631,1397


In [180]:
useful_data_df = useful_data_df.drop(['Book-Author', 'City', 'State'], axis=1)

In [181]:
useful_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031175 entries, 0 to 1031174
Data columns (total 6 columns):
Age                    1031175 non-null float64
Year-Of-Publication    1031175 non-null int64
Book-Rating            1031175 non-null int64
Book-Author-Enc        1031175 non-null int64
City-Enc               1031175 non-null int64
State-Enc              1031175 non-null int64
dtypes: float64(1), int64(5)
memory usage: 55.1 MB


In [182]:
useful_data_df = useful_data_df.apply(lambda x : x.astype(float))
useful_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1031175 entries, 0 to 1031174
Data columns (total 6 columns):
Age                    1031175 non-null float64
Year-Of-Publication    1031175 non-null float64
Book-Rating            1031175 non-null float64
Book-Author-Enc        1031175 non-null float64
City-Enc               1031175 non-null float64
State-Enc              1031175 non-null float64
dtypes: float64(6)
memory usage: 55.1 MB


In [183]:
X, y = useful_data_df.drop('Book-Rating', axis=1), useful_data_df[['Book-Rating']]

In [184]:
X.head()

,Age,Year-Of-Publication,Book-Author-Enc,City-Enc,State-Enc
0,37.397719,2002.0,61695.0,13340.0,1839.0
1,23.000000,2002.0,61695.0,2631.0,1397.0
2,23.000000,1986.0,73676.0,2631.0,1397.0
3,23.000000,1994.0,93473.0,2631.0,1397.0
4,23.000000,1996.0,80512.0,2631.0,1397.0


In [185]:
y.head()

,Book-Rating
0,0.0
1,5.0
2,9.0
3,8.0
4,9.0


In [199]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [200]:
RUN_NAME = calendar.timegm(time.gmtime())

learning_rate = 0.001
training_epochs = 100

number_of_inputs = 5
number_of_outputs = 1

layer_1_nodes = 20
layer_2_nodes = 100
layer_3_nodes = 50

In [201]:
tf.reset_default_graph()

### Input Layer

In [202]:
input_x = tf.placeholder(tf.float64, shape=(None, number_of_inputs), name='Input_X')

### Layer 1

In [203]:
weights_1 = tf.get_variable(name="weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_1 = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_1_output = tf.nn.relu(tf.matmul(input_x, weights_1) + biases_1)

### Layer 2

In [204]:
weights_2 = tf.get_variable(name="weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_2 = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights_2) + biases_2)

### Layer 3

In [205]:
weights_3 = tf.get_variable(name="weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_3 = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer(), dtype=tf.float64)
layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights_3) + biases_3)

### Output

In [206]:
weights_o = tf.get_variable("weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer(), dtype=tf.float64)
biases_o = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer(), dtype=tf.float64)
prediction = tf.matmul(layer_3_output, weights_o) + biases_o

### Cost

In [207]:
actual_y = tf.placeholder(tf.float64, shape=(None, 1), name="Actual_Y")
cost = tf.reduce_mean(tf.squared_difference(prediction, actual_y))

### Optimizer

In [208]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### Logging

In [209]:
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

### Session Initialization

In [214]:
with tf.Session() as session:
    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter("./logs/{}/training".format(RUN_NAME), session.graph)
    testing_writer = tf.summary.FileWriter("./logs/{}/testing".format(RUN_NAME), session.graph)

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())
    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):
        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={input_x: X_train, actual_y: y_train})
        if epoch % 5 == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary],
                                                          feed_dict={input_x: X_train, actual_y: y_train})
            testing_cost, testing_summary = session.run([cost, summary],
                                                        feed_dict={input_x: X_test, actual_y: y_test})
            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)
            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))
    testing_cost = session.run(optimizer, feed_dict={input_x: X_test, actual_y: y_test})
    print("Testing Cost: {}".format(testing_cost))

        

Epoch: 0 - Training Cost: 52600116.024166316  Testing Cost: 52561260.420814686
Epoch: 5 - Training Cost: 3472202.3469564836  Testing Cost: 3466954.394958142
Epoch: 10 - Training Cost: 4025525.1731337244  Testing Cost: 4026498.3261832935
Epoch: 15 - Training Cost: 4748177.093609874  Testing Cost: 4750452.636382547
Epoch: 20 - Training Cost: 1210353.3057232595  Testing Cost: 1211726.8542176366
Epoch: 25 - Training Cost: 39858.60572382683  Testing Cost: 39808.38123903959
Epoch: 30 - Training Cost: 547794.327053852  Testing Cost: 546494.023310575
Epoch: 35 - Training Cost: 529996.1062394353  Testing Cost: 528814.442603361
Epoch: 40 - Training Cost: 112006.69783149363  Testing Cost: 111652.85406996516
Epoch: 45 - Training Cost: 36083.94397642189  Testing Cost: 36244.27415371555
Epoch: 50 - Training Cost: 114258.76303538057  Testing Cost: 114519.94544110981
Epoch: 55 - Training Cost: 63064.01794409779  Testing Cost: 63248.58411881069
Epoch: 60 - Training Cost: 12230.799094223572  Testing Cos

In [211]:
! tensorboard --logdir "./logs"

/usr/local/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
TensorBoard 0.4.0 at http://domino-run-5e8f0bdec9e77c000c8e2ca6:6006 (Press CTRL+C to quit)
^C
